In [13]:
import pandas as pd
import numpy as np
import catboost as cb
from sklearn.model_selection import train_test_split

# 数据加载
Train_data = pd.read_csv('./train_data.csv', index_col=0)
Test_data = pd.read_csv('./test_data.csv', index_col=0)
Test_data.drop('Attrition', axis=1, inplace=True)
x_train, x_test, y_train, y_test = train_test_split(Train_data.drop('Attrition', axis=1), Train_data['Attrition'], test_size=0.2)

In [14]:
attr = ['Age', 'BusinessTravel', 'Department',
       'DistanceFromHome', 'Education', 
       'EnvironmentSatisfaction', 'JobInvolvement',
       'JobLevel', 'JobRole', 'JobSatisfaction', 'MaritalStatus', 'NumCompaniesWorked', 'OverTime',
       'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction',
       'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear',
       'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole',
       'YearsSinceLastPromotion', 'YearsWithCurrManager']

In [15]:
model = cb.CatBoostClassifier(iterations=2000, 
                             depth=7,
                             learning_rate=0.01, 
                             loss_function='Logloss',
                             eval_metric='AUC',
                             logging_level='Verbose',
                             metric_period=50)

In [16]:
categorical_features_indices = []
for i in range(len(x_train.columns)):
    if x_train.columns.values[i] in attr:
        categorical_features_indices.append(i)
print(categorical_features_indices)

[0, 1, 3, 4, 5, 7, 10, 11, 12, 13, 14, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]


In [22]:
model.fit(x_train, y_train, eval_set=(x_test, y_test), cat_features=categorical_features_indices)

0:	test: 0.6237055	best: 0.6237055 (0)	total: 15.9ms	remaining: 31.8s
50:	test: 0.8564725	best: 0.8564725 (50)	total: 961ms	remaining: 36.7s
100:	test: 0.8687702	best: 0.8687702 (100)	total: 2.02s	remaining: 38s
150:	test: 0.8768608	best: 0.8768608 (150)	total: 3.13s	remaining: 38.4s
200:	test: 0.8841424	best: 0.8841424 (200)	total: 4.49s	remaining: 40.1s
250:	test: 0.8823625	best: 0.8841424 (200)	total: 5.76s	remaining: 40.2s
300:	test: 0.8752427	best: 0.8841424 (200)	total: 6.97s	remaining: 39.4s
350:	test: 0.8734628	best: 0.8841424 (200)	total: 8.47s	remaining: 39.8s
400:	test: 0.8729773	best: 0.8841424 (200)	total: 9.85s	remaining: 39.3s
450:	test: 0.8699029	best: 0.8841424 (200)	total: 11.2s	remaining: 38.3s
500:	test: 0.8695793	best: 0.8841424 (200)	total: 12.5s	remaining: 37.3s
550:	test: 0.8686084	best: 0.8841424 (200)	total: 13.9s	remaining: 36.6s
600:	test: 0.8686084	best: 0.8841424 (200)	total: 16.5s	remaining: 38.4s
650:	test: 0.8700647	best: 0.8841424 (200)	total: 19.2s	re

In [23]:
predict = model.predict(Test_data)

In [24]:
temp = pd.read_csv('./test.csv')['user_id']
# print(temp)
submit = pd.DataFrame(temp)
submit['Attrition'] = predict
submit['Attrition'] = submit['Attrition'].map(lambda x:1 if x>=0.5 else 0)
print(submit)
submit.to_csv('submit_cb.csv', index=None)

     user_id  Attrition
0        442          0
1       1091          0
2        981          0
3        785          0
4       1332          1
..       ...        ...
289     1439          0
290      481          0
291      124          0
292      198          0
293     1229          0

[294 rows x 2 columns]
